# Projet SAM -> Classifieur de genre musical

### Treating mxm data, joining datasets (mxm -> msd data)

In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import imutils
from skimage.feature import hog
from pathlib import Path
#import sqlite3

In [2]:
mxm_path = "C:/AMU/SAM/Projet/msdi/mxm/"

## Pre-processing
Creating pandas files for training and test
- Bag of words: < word idx> : < cnt>

In [ ]:
def open_file(file_name):
    with open(mxm_path+file_name) as f:
        lines = f.read().splitlines()
    return lines

In [ ]:
def uncompress_bag(document):
    bag = [0]*2000    
    for word in document:
        w_freq = word.split(":")
        #print(w_freq[0])
        if int(w_freq[0]) <= 2000:
            bag[int(w_freq[0])-1] = int(w_freq[1])
    return bag


In [ ]:
def create_sparse_bag(files):
    document_list = []
    check = 0
    for file in files:
        document = file.split(",")
        tag = document[0:2]
        bow = document[2:len(document)]
        bow = uncompress_bag(bow)
        document = tag + bow
        document_list.append(document)
        if check % 5000 == 0:
            print(check)
        check += 1
    return document_list

In [ ]:
def list2pandas(document_list,filename):
    header = ["mdi_id","mxm_id"] + list(range(1,2001))
    df = pd.DataFrame(document_list,columns = header)
    df.to_csv(mxm_path+filename+".csv")

In [ ]:
file_names = ['mxm_train','mxm_test'] #

In [ ]:
for file_name in file_names:
    files = open_file(file_name+'.txt')
    #print(len(files))
    #print(files[0])
    document_list = create_sparse_bag(files)
    list2pandas(document_list,file_name)

### Joining train end test Files

In [ ]:
mxm_train = pd.read_csv('./msdi/mxm/mxm_train.csv')
mxm_test = pd.read_csv('./msdi/mxm/mxm_test.csv')
mxm_all = pd.concat([mxm_train, mxm_test], ignore_index=True)

In [ ]:
mxm_all.drop(mxm_all.columns[0], axis=1, inplace=True)

In [ ]:
mxm_all.to_csv(mxm_path+"mxm_all.csv")

### Linking with mdi data (class)

In [ ]:
# load the data into a Pandas DataFrame
mxm_df = pd.read_csv('./msdi/mxm/mxm_all.csv')
entries = pd.read_csv("./msdi/msdi_mapping.csv")
# write the data to a sqlite table
#mxm_df.to_sql('mxm', conn, if_exists='append', index = False)

In [35]:
matched_mxm = []
for index, row in entries.iterrows():
    
    if (mxm_df['mdi_id'] == row['msd_track_id']).any():
        row_mxm = mxm_df.loc[mxm_df['mdi_id'] == row['msd_track_id']]
        row_mxm = row_mxm.values.flatten().tolist()
        values = row_mxm[3:len(row_mxm)]
        new_entry = [row['msd_track_id'],row['genre'],row['set']] + values
        matched_mxm.append(new_entry) # Only save the lyrics that have an image/ audio sample

    else:
        entries = entries.drop([index]) # Remove entries that do not have available lyrics
        

In [46]:
entries.drop(entries.columns[0], axis=1, inplace=True)

In [47]:
entries

,msd_track_id,genre,album_index,set,msd_artist_id,image_url,img,mfcc,deep_features
2,TRDMMDE128F14A9052,Rock,2,train,ARJVTRE1187B9959C0,http://artwork-cdn.7static.com/static/img/slee...,img/0000062345_200.jpg,mfcc/mfcc_D.npz,2
3,TRJOPZB128F4250E02,Rock,4,train,AR62BB21187B9AC83D,http://artwork-cdn.7static.com/static/img/slee...,img/0000144785_200.jpg,mfcc/mfcc_J.npz,3
4,TRJKBVL128F935567B,Rock,5,train,AR7GVOV1187B9B5FF1,http://artwork-cdn.7static.com/static/img/slee...,img/0000520513_200.jpg,mfcc/mfcc_J.npz,4
5,TRUSOQD128F92CFEA5,Rock,7,train,ARZHQDD1187FB5B871,http://artwork-cdn.7static.com/static/img/slee...,img/0000318842_200.jpg,mfcc/mfcc_U.npz,5
6,TRKCEWE128F425BF56,Rock,9,train,ARRSEGJ1187B9A67D9,http://artwork-cdn.7static.com/static/img/slee...,img/0000278069_200.jpg,mfcc/mfcc_K.npz,6
...,...,...,...,...,...,...,...,...,...
30707,TRJRMVM128F4259C12,Punk,12066,test,ARC1SV21187FB4CF26,http://artwork-cdn.7static.com/static/img/slee...,img/0000250669_200.jpg,mfcc/mfcc_J.npz,4644
30708,TRJUGOK128F4243DA6,Pop,22364,test,AR5MK521187B98E0B8,http://artwork-cdn.7static.com/static/img/slee...,img/0000139188_200.jpg,mfcc/mfcc_J.npz,4645
30709,TRTMWUS128F425D9AF,Pop,8309,test,ARY2W5X1187FB3CB7A,http://artwork-cdn.7static.com/static/img/slee...,img/0000150164_200.jpg,mfcc/mfcc_T.npz,4646
30710,TRSBAKM128F931C033,Rap,16326,test,ARY47DB1187B9A801C,http://artwork-cdn.7static.com/static/img/slee...,img/0001059105_200.jpg,mfcc/mfcc_S.npz,4647


In [44]:
matched_df = pd.DataFrame(matched_mxm,columns = (["mdi_id",'genre','set'] + list(range(1,2001))))

In [1]:
matched_df 

NameError: name 'matched_df' is not defined

In [48]:
matched_df.to_csv(mxm_path+"joint_mxm"+".csv")
entries.to_csv(mxm_path+"joint_msdi"+".csv")

### Save joint datasets (image/ audio - text)

In [4]:
msdi_joint = pd.read_csv('./msdi/mxm/joint_msdi.csv')

In [5]:
msdi_joint

,Unnamed: 0,msd_track_id,genre,album_index,set,msd_artist_id,image_url,img,mfcc,deep_features
0,2,TRDMMDE128F14A9052,Rock,2,train,ARJVTRE1187B9959C0,http://artwork-cdn.7static.com/static/img/slee...,img/0000062345_200.jpg,mfcc/mfcc_D.npz,2
1,3,TRJOPZB128F4250E02,Rock,4,train,AR62BB21187B9AC83D,http://artwork-cdn.7static.com/static/img/slee...,img/0000144785_200.jpg,mfcc/mfcc_J.npz,3
2,4,TRJKBVL128F935567B,Rock,5,train,AR7GVOV1187B9B5FF1,http://artwork-cdn.7static.com/static/img/slee...,img/0000520513_200.jpg,mfcc/mfcc_J.npz,4
3,5,TRUSOQD128F92CFEA5,Rock,7,train,ARZHQDD1187FB5B871,http://artwork-cdn.7static.com/static/img/slee...,img/0000318842_200.jpg,mfcc/mfcc_U.npz,5
4,6,TRKCEWE128F425BF56,Rock,9,train,ARRSEGJ1187B9A67D9,http://artwork-cdn.7static.com/static/img/slee...,img/0000278069_200.jpg,mfcc/mfcc_K.npz,6
...,...,...,...,...,...,...,...,...,...,...
14745,30707,TRJRMVM128F4259C12,Punk,12066,test,ARC1SV21187FB4CF26,http://artwork-cdn.7static.com/static/img/slee...,img/0000250669_200.jpg,mfcc/mfcc_J.npz,4644
14746,30708,TRJUGOK128F4243DA6,Pop,22364,test,AR5MK521187B98E0B8,http://artwork-cdn.7static.com/static/img/slee...,img/0000139188_200.jpg,mfcc/mfcc_J.npz,4645
14747,30709,TRTMWUS128F425D9AF,Pop,8309,test,ARY2W5X1187FB3CB7A,http://artwork-cdn.7static.com/static/img/slee...,img/0000150164_200.jpg,mfcc/mfcc_T.npz,4646
14748,30710,TRSBAKM128F931C033,Rap,16326,test,ARY47DB1187B9A801C,http://artwork-cdn.7static.com/static/img/slee...,img/0001059105_200.jpg,mfcc/mfcc_S.npz,4647


## Fetch joint images, audio and DF info

In [6]:
msdi_images = list(msdi_joint['img'])
msdi_mfcc = list(msdi_joint['mfcc'])
msdi_df_info = msdi_joint[["msd_track_id",'genre','set']] #df1 = df[['a', 'b']]

#### Read images

In [7]:
def load_img(img_path):
    #img_path = msdi_path+entry['img']
    if os.path.exists(img_path):
        return cv2.imread(img_path)
    else:
        return 0

In [11]:
remove_index = []

#### Apply HOG

In [12]:
def hog_through(image_set):
    file_list = []
    for i in range(len(image_set)):
        if i%1000 == 0:
            print("check point:",i)
        
        img = load_img('./msdi/'+image_set[i])

        if img.any():
            
            #resized_img = imutils.resize(img, width=100, height=100)
            fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1),
                            visualize=True,
                            feature_vector=True, channel_axis = 2) # channel_axis = 2 pour les images colores
            
            fd = fd.tolist()
            #print(len(fd))
            file_list.append(fd)
        else:
            fd = np.empty(1152)
            fd[:]= np.NaN
            file_list.append(fd)
            remove_index.append(i)
    return pd.DataFrame.from_records(file_list)
        

In [13]:
joint_hog = hog_through(msdi_images)

check point: 0
check point: 1000
check point: 2000
check point: 3000
check point: 4000
check point: 5000
check point: 6000
check point: 7000
check point: 8000
check point: 9000
check point: 10000
check point: 11000
check point: 12000
check point: 13000
check point: 14000


In [14]:
remove_index

[6500, 7633]

In [16]:
joint_hog

,0,1,2,3,4,5,6,7,8,9,...,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151
0,0.461526,0.095079,0.359829,0.461526,0.461526,0.060185,0.076390,0.461526,0.352322,0.400705,...,0.375861,0.248141,0.405823,0.359692,0.405823,0.213287,0.405823,0.405823,0.405823,0.040859
1,0.353553,0.353553,0.353553,0.353553,0.353553,0.353553,0.353553,0.353553,0.358752,0.355191,...,0.397110,0.164228,0.156867,0.321545,0.433843,0.433843,0.433843,0.433843,0.331413,0.096369
2,0.437474,0.346784,0.437474,0.437474,0.437474,0.247261,0.138914,0.183767,0.353553,0.353553,...,0.375942,0.270150,0.396579,0.396579,0.396579,0.396579,0.396579,0.343110,0.225956,0.211765
3,0.089707,0.316462,0.177347,0.461417,0.461417,0.450414,0.414974,0.243853,0.469203,0.186643,...,0.095449,0.031053,0.029097,0.043288,0.029686,0.575177,0.575177,0.571130,0.090445,0.019272
4,0.149112,0.200487,0.485887,0.538063,0.538063,0.339789,0.072695,0.041477,0.226660,0.245898,...,0.379806,0.205987,0.172742,0.211458,0.375791,0.414679,0.414679,0.414679,0.414679,0.310470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14745,0.552385,0.224374,0.092028,0.552385,0.552385,0.124418,0.012098,0.100855,0.022068,0.000000,...,0.252776,0.534422,0.299088,0.274425,0.327976,0.066294,0.238392,0.556651,0.556651,0.216156
14746,0.473143,0.355811,0.123497,0.332479,0.473143,0.025103,0.274557,0.473143,0.406789,0.406789,...,0.388078,0.388078,0.353553,0.353553,0.353553,0.353553,0.353553,0.353553,0.353553,0.353553
14747,0.376856,0.376856,0.376856,0.310611,0.376856,0.226708,0.376856,0.376856,0.412941,0.412941,...,0.276603,0.363982,0.314840,0.379139,0.379139,0.379139,0.379139,0.379139,0.259972,0.338462
14748,0.484869,0.180924,0.129718,0.484869,0.484869,0.037432,0.092987,0.484869,0.040006,0.012988,...,0.412827,0.412827,0.370777,0.370777,0.370777,0.370777,0.370777,0.278068,0.370777,0.312766


In [17]:
joint_img= pd.concat([msdi_df_info, joint_hog], axis=1)

In [18]:
joint_img.to_csv('./msdi/mxm/joint_hog.csv',index=False)

### Save joint Audio

In [19]:
msdi_path = './msdi/' #msdi_mfcc

In [20]:
def load_mfcc(entry, msdi_path=msdi_path):
    x = np.load(Path(msdi_path) / entry['mfcc'])
    return x[entry['msd_track_id']]

In [21]:
def fetch_mfccs():
    mfcc_list = []
    for index, row in msdi_joint.iterrows():

        mfcc = load_mfcc(row)
        if len(mfcc) >= 500:
            mfcc = mfcc[:500]
            mfcc = [item for sublist in mfcc for item in sublist]
            mfcc_list.append(mfcc)
        elif len(mfcc) < 500:
            mfcc = np.empty(6000)
            mfcc[:]= np.NaN
            mfcc_list.append(mfcc)
            remove_index.append(index)
    return pd.DataFrame.from_records(mfcc_list)

In [22]:
joint_mfcc = fetch_mfccs()

In [24]:
joint_mfcc

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,0.057,169.920,7.709,-29.802,56.171,-49.940,15.719,4.642,-27.179,1.640,...,4.161,8.815,-69.617,-37.691,-13.511,-13.706,11.932,10.410,-6.348,2.725
1,0.045,170.354,7.492,-29.370,55.831,-50.155,16.650,5.226,-26.784,1.888,...,47.062,-27.600,-14.829,-6.559,17.040,-10.554,0.908,16.878,12.868,-7.954
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000,171.130,9.469,-28.480,57.491,-50.067,14.833,5.359,-27.228,0.973,...,24.124,21.388,-25.656,-9.503,9.170,-18.722,5.710,-19.380,3.094,-16.526
4,0.793,157.260,-25.486,-39.031,29.367,-52.463,41.024,9.288,-19.485,11.724,...,23.388,-13.395,-19.751,-32.469,4.947,-5.303,-2.133,4.288,0.086,-7.512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14745,0.000,171.130,9.469,-28.480,57.491,-50.067,14.833,5.359,-27.228,0.973,...,-0.470,-4.745,-12.223,-21.543,3.541,1.682,19.860,10.236,-0.002,12.140
14746,0.000,171.130,9.469,-28.480,57.491,-50.067,14.833,5.359,-27.228,0.973,...,5.989,-22.123,18.942,-28.259,12.842,13.560,20.686,-21.917,3.645,28.129
14747,0.044,170.367,7.566,-29.460,55.842,-50.246,16.657,5.039,-26.776,1.844,...,54.890,-1.725,3.751,-6.600,-24.716,1.550,34.541,15.368,-1.572,-11.735
14748,31.943,-98.771,14.145,-70.102,19.477,161.949,53.817,28.659,-25.708,108.505,...,-18.013,17.458,-7.050,7.263,18.050,-24.836,8.035,28.546,4.534,-10.392


In [25]:
joint_audio= pd.concat([msdi_df_info, joint_mfcc], axis=1)

In [26]:
joint_audio

,msd_track_id,genre,set,0,1,2,3,4,5,6,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,TRDMMDE128F14A9052,Rock,train,0.057,169.920,7.709,-29.802,56.171,-49.940,15.719,...,4.161,8.815,-69.617,-37.691,-13.511,-13.706,11.932,10.410,-6.348,2.725
1,TRJOPZB128F4250E02,Rock,train,0.045,170.354,7.492,-29.370,55.831,-50.155,16.650,...,47.062,-27.600,-14.829,-6.559,17.040,-10.554,0.908,16.878,12.868,-7.954
2,TRJKBVL128F935567B,Rock,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRUSOQD128F92CFEA5,Rock,train,0.000,171.130,9.469,-28.480,57.491,-50.067,14.833,...,24.124,21.388,-25.656,-9.503,9.170,-18.722,5.710,-19.380,3.094,-16.526
4,TRKCEWE128F425BF56,Rock,train,0.793,157.260,-25.486,-39.031,29.367,-52.463,41.024,...,23.388,-13.395,-19.751,-32.469,4.947,-5.303,-2.133,4.288,0.086,-7.512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14745,TRJRMVM128F4259C12,Punk,test,0.000,171.130,9.469,-28.480,57.491,-50.067,14.833,...,-0.470,-4.745,-12.223,-21.543,3.541,1.682,19.860,10.236,-0.002,12.140
14746,TRJUGOK128F4243DA6,Pop,test,0.000,171.130,9.469,-28.480,57.491,-50.067,14.833,...,5.989,-22.123,18.942,-28.259,12.842,13.560,20.686,-21.917,3.645,28.129
14747,TRTMWUS128F425D9AF,Pop,test,0.044,170.367,7.566,-29.460,55.842,-50.246,16.657,...,54.890,-1.725,3.751,-6.600,-24.716,1.550,34.541,15.368,-1.572,-11.735
14748,TRSBAKM128F931C033,Rap,test,31.943,-98.771,14.145,-70.102,19.477,161.949,53.817,...,-18.013,17.458,-7.050,7.263,18.050,-24.836,8.035,28.546,4.534,-10.392


In [27]:
joint_audio.to_csv('./msdi/mxm/joint_mfcc.csv',index=False)

### Cut off Samples with Nan from all three datasets simultaneusly 
     (keep data joint)

In [28]:
text_data = pd.read_csv('./msdi/mxm/joint_mxm.csv')
img_data = pd.read_csv('./msdi/mxm/joint_hog.csv')
audio_data = pd.read_csv('./msdi/mxm/joint_mfcc.csv')

In [32]:
len(remove_index)

1583

In [29]:
def fetch_any_null_rows(df):
    is_NaN = df.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = df[row_has_NaN]
    return rows_with_NaN

In [47]:
nulls_txt = fetch_any_null_rows(text_data)
nulls_img = fetch_any_null_rows(img_data)
nulls_audio = fetch_any_null_rows(audio_data)

In [48]:
nulls = [nulls_txt,nulls_img,nulls_audio]

In [59]:
for null in nulls:
    try:
        text_data.drop(null, axis=0 ,inplace=True)
    except:
        pass
    try:
        img_data.drop(null, axis= 0,inplace=True)
    except:
        pass
    try:
        audio_data.drop(null, axis= 0, inplace=True)
    except:
        pass
        

In [60]:
print(len(text_data),len(img_data),len(audio_data))

14750 14750 14750


### Cut Samples which the audio has a mfcc with a size < 500 

In [36]:
text_data.drop(remove_index, inplace=True)
img_data.drop(remove_index, inplace=True)
audio_data.drop(remove_index, inplace=True)

In [37]:
print(len(text_data),len(img_data),len(audio_data))

13168 13168 13168


In [39]:
text_data.to_csv('./msdi/mxm/joint_mxm.csv',index=False)
img_data.to_csv('./msdi/mxm/joint_hog.csv',index=False)
audio_data.to_csv('./msdi/mxm/joint_audio.csv',index=False)